In [17]:
import requests
import pandas as pd
from tqdm import tqdm
from newspaper import Article

In [15]:
all_data = []
for i in tqdm(range(100), desc="Fetching Articles"):
    url = f"https://www.reuters.com/pf/api/v3/content/fetch/articles-by-section-alias-or-id-v1?query=%7B%22arc-site%22%3A%22reuters%22%2C%22called_from_a_component%22%3Atrue%2C%22fetch_type%22%3A%22sophi%22%2C%22offset%22%3A{i}%2C%22section_id%22%3A%22%2Fworld%2F%22%2C%22size%22%3A9%2C%22sophi_page%22%3A%22*%22%2C%22sophi_widget%22%3A%22topic%22%2C%22uri%22%3A%22%2Fworld%2F%22%2C%22website%22%3A%22reuters%22%7D&d=153&_website=reuters"
    response = requests.get(url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        if 'result' in data and 'articles' in data['result']:
            all_data.extend(data['result']['articles'])

Fetching Articles: 100%|██████████| 100/100 [01:25<00:00,  1.17it/s]


In [16]:
# Create a DataFrame from the list of articles
df = pd.DataFrame(all_data)

# Print the first few rows of the DataFrame
print(df.shape)
links = df['canonical_url']
print(links)

(864, 19)
0      /world/us/maui-tourism-an-economic-mainstay-sp...
1      /world/russian-raid-off-turkeys-coast-tests-er...
2      /world/europe/cargo-ship-leaves-ukrainian-port...
3      /world/europe/ukraine-says-recaptures-urozhain...
4      /world/europe/wildfire-tenerife-national-park-...
                             ...                        
859    /world/middle-east/jordan-army-downs-drone-loa...
860    /world/europe/ukraines-frontline-farmers-face-...
861    /world/middle-east/jordan-army-downs-drone-loa...
862    /world/europe/ukraines-frontline-farmers-face-...
863    /world/europe/ukraines-frontline-farmers-face-...
Name: canonical_url, Length: 864, dtype: object


In [19]:
articles = {
    'Title': [],
    'Description': []
}
URL='https://www.reuters.com'
for index, row in tqdm(df.iterrows(), total=len(df)):
    link = URL + row['canonical_url']
    news_article = Article(link)
    news_article.download()
    news_article.parse()
    title = news_article.title
    description = news_article.text
    articles['Title'].append(title)
    articles['Description'].append(description)

100%|██████████| 864/864 [11:55<00:00,  1.21it/s]


In [20]:
news_df = pd.DataFrame(articles)
news_df['Label'] = 1
news_df.to_csv('reuters_news.csv',index=False)